In [1]:
%%time
%%sh

# The name of our algorithm
# Makoto.Sano@Mack-the-Psych.com
algorithm_name=sagemaker-vdok3-memn2n

cd container

chmod +x vdok3_sage/train
chmod +x vdok3_sage/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
Sending build context to Docker daemon   42.5kB
Step 1/16 : FROM continuumio/anaconda3:4.4.0
 ---> 795ad88c47ff
Step 2/16 : RUN conda install -c anaconda setuptools
 ---> Using cache
 ---> 7646ff61d3af
Step 3/16 : RUN apt-get -y update && apt-get install -y --no-install-recommends          libgl1-mesa-glx          wget          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 420cd5bb794e
Step 4/16 : RUN pip install --upgrade pip &&     pip install janome==0.3.10 &&     pip install tensorflow==1.14.0 --user &&     pip install ml_metrics==0.1.4 &&     pip install --upgrade scipy==1.1.0 &&     pip install flask &&     pip install -U gevent==1.4.0 --ignore-installed &&     pip install gunicorn
 ---> Using cache
 ---> 23a5ffac18cd
Step 5/16 : RUN python -m nltk.downloader book
 ---> Using cache
 ---> 4db5f1677ff0
Step 6/16 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 3171e306868d
Step 7/16 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 -

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



CPU times: user 8.84 ms, sys: 154 µs, total: 8.99 ms
Wall time: 8 s


In [2]:
# S3 prefix
# Makoto.Sano@Mack-the-Psych.com
prefix = 'vdok3_memn2n_trial'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [3]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [4]:
WORK_DIRECTORY = 'data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

In [5]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

# Makoto.Sano@Mack-the-Psych.com
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-vdok3-memn2n:latest'.format(account, region)
vdok3memn2n = sage.estimator.Estimator(image,
                       role, 1, 'ml.c4.2xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)
vdok3memn2n.fit(data_location)

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-09-13 09:01:47 Starting - Starting the training job...
2020-09-13 09:01:49 Starting - Launching requested ML instances......
2020-09-13 09:03:05 Starting - Preparing the instances for training...
2020-09-13 09:03:46 Downloading - Downloading input data...
2020-09-13 09:03:52 Training - Downloading the training image......../root/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/root/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/root/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:

In [6]:
from sagemaker.predictor import csv_serializer
predictor = vdok3memn2n.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


-----------------!

In [10]:
df_in = pd.read_csv('data/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv')
np_in = np.vstack((np.array(df_in.columns), df_in.to_numpy()))
print(predictor.predict(np_in).decode('utf-8'))

Score_Class
1
1
2
0



In [11]:
sess.delete_endpoint(predictor.endpoint)